# TD - Word embedding and RNN network

## 0. The different imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [ ]:
from keras.utils import pad_sequences
from keras import models, layers, callbacks

In [ ]:
try:
    import gensim
except:
    !pip install gensim
    import gensim

In [ ]:
import os
from gensim.models import KeyedVectors
import gensim.downloader as api # from

_EMBEDDING_SIZE = 50

# Download if not exists
if not os.path.exists("glove-wiki-gigaword-"+str(_EMBEDDING_SIZE)+".kv"):
    print("Download")
    word_vectors = api.load("glove-wiki-gigaword-"+str(_EMBEDDING_SIZE))
    word_vectors.save("glove-wiki-gigaword-"+str(_EMBEDDING_SIZE)+".kv")
else:
    # Load the model
    print("Load")
    word_vectors = KeyedVectors.load("glove-wiki-gigaword-"+str(_EMBEDDING_SIZE)+".kv", mmap='r')

# fix vocabulary size
_VOCABULARY_SIZE = len(word_vectors.key_to_index)
_VOCABULARY_SIZE

## 1. The magic of good embedding

In [ ]:
# Get the embedding of a word
word_vectors['university']  

In [ ]:
# Find most similar words
word_vectors.most_similar('university')

**Un peu de calcul avec les vecteurs**
![woman-man+king](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTDeOAaVruIywX9lhjqdfWZ70iSMTYX-3eW5g&s)

In [ ]:
# Playing with vectors
word_vectors.most_similar(positive=['woman', 'king'], negative=['man'])

**En voici un autre**

In [ ]:
word_vectors.most_similar(positive=['rome', 'france'], negative=['italy'])

In [ ]:
""" Essayer de trouver d'autres triplets qui fonctionnent et publiez les sur slack """

In [ ]:
# Find the hidden word
word_vectors.doesnt_match("breakfast cereal dinner lunch".split())

In [ ]:
# Find the hidden word
word_vectors.doesnt_match("monday tuesday friday sunday".split())

In [ ]:
""" Essayer de trouver d'autres ensemble de mots avec un mot incongru bien détecté et publiez le sur slack """

## 2. Build a RNN for a NER Task

NER seeks to extract and classify words into predefined categories such as person names, organizations, locations, medical codes, time expressions, quantities, monetary values, etc.

![NER task](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*7Rhj-zxvJGG_Pw7cQSoa6w.png)

### 2.1 Prepare the dataset

In [ ]:
data = pd.read_csv("ner_dataset.csv", encoding="latin1")
data = data.fillna(method="ffill")
data.head(20)

#### Retrieve sentence

In [ ]:
class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None


In [ ]:
getter = SentenceGetter(data)
sentences = getter.sentences

In [ ]:
sentences[0]

In [ ]:
# Split dataset before working

TRAIN, TEST = train_test_split(sentences, test_size=0.2, random_state=42)
TRAIN, VAL = train_test_split(TRAIN, test_size=0.1, random_state=42)

### 2.2 A Simple recurrent neural network for NER task

![RNN for NER](https://confusedcoders.com/wp-content/uploads/2019/12/many_to_many-1024x530.png)

* Red: embedding layer
* Green: recurrent cells
* Blue: dense cells

La manière la plus simple de procéder est de considérer que nous avons à notre disposition uniquement les données d'entrainement et que nous utilisons la couche d'embedding de Keras. Néanmoins, un mot doit être représenté par un nombre. Commençon donc par transformer chacun des mots de nos données d'entrainement par un nombre.

In [ ]:
# Define vocabulary from the train part
words = ["<PAD>", "<UNK>"]+sorted(list({w[0] for s in TRAIN for w in s}))
_NUM_WORDS = len(words)
print("Unique words in corpus:", _NUM_WORDS)

word2idx = {w: i for i, w in enumerate(words)}
idx2word = {v:k for k, v in word2idx.items()}

words[:5]

In [ ]:
# Define tags from the train part

tags = ["<PAD>"]+sorted(list({w[2] for s in TRAIN for w in s}),reverse=True)
_NUM_TAGS = len(tags)
print("Unique tags in corpus:", _NUM_TAGS)

tag2idx = {w: i for i, w in enumerate(tags)}
idx2tag = {v:k for k, v in tag2idx.items()}

tags[:5]

In [ ]:
# Max_len without truncation for the train

_MAX_LEN = max([len(sent) for sent in TRAIN])
_MAX_LEN

In [ ]:
X_train = pad_sequences(maxlen=_MAX_LEN,
                        sequences=[[word2idx[w[0]] if word2idx.get(w[0]) is not None else word2idx["<UNK>"] for w in s] for s in TRAIN],
                        padding="pre", truncating="post",
                        value=word2idx["<PAD>"])
                        # padding="pre" and truncating="pre" by default

y_train = pad_sequences(maxlen=_MAX_LEN,
                        sequences=[[tag2idx[w[2]] for w in s] for s in TRAIN],
                        padding="pre", truncating="post",
                        value=tag2idx["<PAD>"])

X_val = pad_sequences(maxlen=_MAX_LEN,
                      sequences=[[word2idx[w[0]] if word2idx.get(w[0]) is not None else word2idx["<UNK>"] for w in s] for s in VAL],
                      padding="pre", truncating="post", value=word2idx["<PAD>"])

y_val = pad_sequences(maxlen=_MAX_LEN,
                        sequences= [[tag2idx[w[2]] for w in s] for s in VAL],
                        padding="pre", truncating="post", value=tag2idx["<PAD>"])

X_test = pad_sequences(maxlen=_MAX_LEN,
                       sequences=[[word2idx[w[0]] if word2idx.get(w[0]) is not None else word2idx["<UNK>"] for w in s] for s in TEST],
                       padding="pre", truncating="post", value=word2idx["<PAD>"])

y_test = pad_sequences(maxlen=_MAX_LEN,
                       sequences=[[tag2idx[w[2]] for w in s] for s in TEST],
                       padding="pre", truncating="post", value=tag2idx["<PAD>"])

#### Un premier réseau RNN

Bon, j'ai fait tout le travail préparatoire pour vous. A vous de jouer et de remplacer les ... par le code correct.

Il est possible d'ajouter du dropout sur la couche récurrente pour limiter l'overfitting.

In [ ]:
_HIDDEN_SIZE = 32
_DROPOUT = 0.4

inputs = layers.Input(shape=(_MAX_LEN,), dtype=int)
emb = layers."..."        # Embedding step
assert emb.shape==(None, 104, 50)

hidden = layers."..."     # Recurrent step
assert hidden.shape==(None, 104, 32)

outputs = layers."..."    # Output step
assert hidden.shape==(None, 104, 17)

model = models.Model(inputs, outputs)
model.summary()

In [ ]:
""" On compile avec la bonne fonction de cout """
model.compile(optimizer="adam", loss=...)

""" On entraine jusqu'au début de l'overfitting """
history = model.fit(X_train, y_train, validation_data=[X_val, y_val], epochs=1000,
          callbacks=[callbacks.EarlyStopping(...)])

In [ ]:
""" on verifie que le réseau apprend mais pas trop vite """

""" Ayant peu de données, le réseau apprend relativement vite en 5-6 epoques """

""" Avez-vous remarqué, que le temps d'apprentissage est nettement plus lent
qu'avec des cellules Dense """

def babysit(history):    
    plt.plot(history['loss'], label="loss")
    plt.plot(history['val_loss'], label="val_loss")
    plt.show()

babysit(history.history)

In [ ]:
""" On évalue les performances en éliminant la classe 'O' omni-présente
et bien évidement le padding """
y_pred = np.argmax(model.predict(X_test), axis=2)
print(classification_report(y_test.flatten(), y_pred.flatten(), zero_division=0.0,
                            target_names=tags[2:], labels=range(2,len(tags)), digits=2))

### 2.3 A bidirectionnal recurrent neural network for NER task

![BiLSTM](https://www.researchgate.net/publication/332375961/figure/fig3/AS:746852529999875@1555074927914/A-biLSTM-network-for-NER-tasks-English-Translation-Taxquenas-Soriana-has-fallen-down.ppm)

In [ ]:
""" A vous de jouer """

In [ ]:
model.compile(optimizer="adam", loss="...")
history = model.fit(X_train, y_train, validation_data=[X_val, y_val], epochs=1000,
          callbacks=[callbacks.EarlyStopping(...)])

In [ ]:
babysit(...)

In [ ]:
# Evaluation without tags 'O' and padding
y_pred = np.argmax(model.predict(X_test), axis=2)
print(classification_report(...))

In [ ]:
""" Est-ce que l'on fait mieux que précédemment,
en particulier pour les classes sous représentées """

### 2.4 A Stacked recurrent neural network for NER task

Même chose mais avec plusieurs couches récurrentes, par exemple une première couche avec un BI-LSTM qui effectue la moyenne des deux sorties (cf. doc Bidirectional layer, parametre merge_mod) et une seconde couche avec un GRU. 

In [ ]:
""" A vous de jouer """

In [ ]:
model.compile(optimizer="adam", loss=...)
history = model.fit(X_train, y_train, validation_data=[X_val, y_val], epochs=1000,
          callbacks=[callbacks.EarlyStopping(...)])

In [ ]:
babysit(...)

In [ ]:
# Suppress tags 'O' and padding
y_pred = np.argmax(model.predict(X_test), axis=2)
print(classification_report(...))

In [ ]:
""" Avez-vous progressé ? Ce n'est pas certain... parfois un réseau simple fonctionne
mieux qu'un réseau trop sophistiqué """

### 2.5 How to use a pre-trained embedding

Si on souhaite utiliser un embedding existant, le plus efficace est d'initialiser la couche d'embedding avec ces poids. Pour cela, il faut redéfinir le mapping mot-id afin qu'il corresponde à celui utilisé pour l'entrainement de l'embedding utilisé.

In [ ]:
# Rappel
# _VOCABULARY_SIZE contient la taille du vocabulaire de l'embedding pré-entrainé utilisé
# word_vectors contient le vocabulaire et les différents vecteurs

In [ ]:
# Redéfinition du mapping mot-id

# On peut obtenir la liste "ordonnée" des mots de la manière suivante
words = ["<PAD>", "<UNK>"]+word_vectors.index_to_key
_NUM_WORDS = len(words)
print("Unique words in corpus:", _NUM_WORDS)

# Les dictionnaires permettant de passer d'un mot à son index (ou l'inverse)
# sont les suivants : 
word2idx = {w: i for i, w in enumerate(words)}
idx2word = {v:k for k, v in word2idx.items()}

words[:5]

In [ ]:
# Redéfinition des vecteurs d'entrainement
X_train = ...

X_val = ...

X_test = ...

# Inutile de modifier y_train, y_val, y_test
# le mapping tag-id n'ayant pas été modifié

In [ ]:
# Définition de la matrice d'embedding
embedding_matrix = np.zeros((_NUM_WORDS, _EMBEDDING_SIZE))

""" reste à initialiser la matrice d'embedding """
A vous de jouer ...

In [ ]:
# Boucle de vérification
import random
for i, r in enumerate([random.randint(0, _NUM_WORDS) for _ in range(20)]):
    assert np.array_equal(word_vectors[idx2word[r]],embedding_matrix[r])

In [ ]:
# Création du reseau - seul changement,
# Pensez à initialiser la couche d'embedding  avec la matrice d'embedding
# Un petit coup d'oeil à la doc Keras Embedding pour trouver le paramètre
""" a vous de jouer """

In [ ]:
model.compile(...)
history = model.fit(...)

In [ ]:
babysit(...)

In [ ]:
# Suppress tags 'O' and padding
y_pred = ...
print(classification_report(...)), digits=2))

In [ ]:
""" Est-ce que l'on fait mieux que dans les cas précédents ?
Si ce n'est pas le cas, il peut être interessant de regarder si le vocabulaire
utilisé dans l'embedding correspond à celui du jeu de données """